In [0]:
from random import randint, randrange, uniform

n_devices = 200 #  with different completion times
required_devices = 0.8
starting_deadline = 1.0
nData = 1000
devices = []

completition_time_model = lambda nCPU,nData,frequency : (nCPU*nData)/frequency
id = 0
class device:
  id = 0
  consumedEnergy = 0.0
  cpuFrequencyMax = 0.0
  optimalFrequency = 0.0
  completitionTime = 0.0
  nCPU = 0
  quality = 0
  global_deadline = 0.0
  beta = 0.0
  power_train = 0.0
  power_idle = 0.0
  energyMap = {}
  EnergyConsumption = 0.0

  def __init__(self,quality,id):
    self.id = id
    self.quality = quality
    self.cpuFrequencyMax = round(self.quality/10 + randrange(10,16,5)/100,3)
    self.nCPU = randint(1,5)
    self.beta = self.quality * randint(1,5) / 10


  def set_deadline(self,deadline):
    self.global_deadline = deadline
  
  # 
  #   Speed Determination
  # 
  def setOptimalCpu(self,nData):
    optimalCpuFrequency = round(self.quality/20 + randrange(10,16,5)/100,3)
    self.energyMap = {}
    while(optimalCpuFrequency < self.cpuFrequencyMax):
      time_idle = 0
      time_train = 0
      self.completitionTime = completition_time_model(self.nCPU,nData,optimalCpuFrequency)
      
      if(self.completitionTime < self.global_deadline):
        self.optimalFrequency = optimalCpuFrequency
        self.computeEnergy()
        self.energyMap[round(optimalCpuFrequency,3)] = self.EnergyConsumption

      optimalCpuFrequency = round(optimalCpuFrequency + 0.1,3)

    if(len(self.energyMap) > 0 ):
      self.optimalFrequency = min(self.energyMap.items(), key=lambda x: x[1])[0]
      self.computeEnergy()
      return 1
    else:
      self.optimalFrequency = self.cpuFrequencyMax
      self.computeEnergy()
      return 0

  # 
  #   set MAX CPU
  # 
  def setCpu(self,nData):
    self.optimalFrequency = self.cpuFrequencyMax
    self.computeEnergy()
    if(self.completitionTime < self.global_deadline):
      return 1
    else:
      return 0


  def computeEnergy(self):
      self.completitionTime = completition_time_model(self.nCPU,nData,self.optimalFrequency)
      time_idle = self.global_deadline - self.completitionTime
      time_train = self.global_deadline - time_idle
      self.power_train = round(self.beta * pow(self.optimalFrequency,3),3)
      self.power_idle = round(self.power_train * 0.1,3)
      self.EnergyConsumption = round(self.power_train * time_train + self.power_idle * time_idle,3)


  def printEnergyMap(self):
    print(str(self.energyMap))


  def __str__(self):
    return "Device ID: "+str(self.id)+" - quality ["+str(self.quality)+ "]: Frequency MAX: " + str(self.cpuFrequencyMax) + " OPTIMAL CPU: " +  str(self.optimalFrequency) + " POWER TRAIN: "+ str(self.power_train) + " POWER IDLE: "+ str(self.power_idle) 
  #Local Pace Control




  
# minimum completition time will be with frequency to the maximum value
# without considering foreground apps
# if we want to consider foreground apps we need to use the completition time prediction formula


def deadline_determination(devices,nData, required_devices,starting_deadline):
  #here we need to find the minimun deadline to satisfy the deadline minimization
  completitionTimes = []
  for device in devices:
    completitionTimes.append(completition_time_model(device.nCPU,nData,device.cpuFrequencyMax))
    indicatorArr = []
    for time in completitionTimes:
      if time < starting_deadline:
        indicatorArr.append(1)
      else:
        indicatorArr.append(0)
  
  while(sum(indicatorArr)/n_devices <= len(devices)*required_devices):
    starting_deadline+=1.0 #di quanto aumentare la deadline ad ogni ciclo
    for time in completitionTimes:
      if time < starting_deadline:
        indicatorArr.append(1)
      else:
        indicatorArr.append(0)
  return starting_deadline

In [38]:
pDevice = 0.5 # Higher means there will be more High-end devices

for i in range(n_devices):
  if(uniform(0,1) < pDevice):
    devices.append(device(randint(8,15),i))
  else:
    devices.append(device(randint(1,8),i)) 

deadlines = []
n_devices_whomadeit = []
devices_ok = [] 
deadline_SP = 0
deadline_classic = 0
global_deadline = deadline_determination(devices,nData,required_devices,starting_deadline)
print("Global deadline based on all the devices is: " + str(global_deadline))
deadlines.append(global_deadline)

iteration = 0
for i in devices:
  # print(i)
  i.set_deadline(global_deadline)
  devices_ok.append(i.setOptimalCpu(nData))

print("For the virtual deadline Devices who can make it: "+str(devices_ok.count(1)) + " VS who can't: " +str(devices_ok.count(0)) )
print("Percentage is: "+str(round(devices_ok.count(1)*100/len(devices_ok),2)) + "% " )
n_devices_whomadeit.append(devices_ok.count(1))
while(devices_ok.count(1) < required_devices * n_devices):
  iteration +=1
  devices_ok = []
  # Recalculate new deadline for next round a bit larger based on old deadline so to avoid useless computation
  global_deadline = deadline_determination(devices,nData,required_devices,global_deadline)
  for i in devices:
    # print(i)
    i.set_deadline(global_deadline)
    devices_ok.append(i.setOptimalCpu(nData))
  print("Devices who can make it after "+str(iteration)+" iteration : "+str(devices_ok.count(1)) + " VS who still can't: " +str(devices_ok.count(0)) )
  deadlines.append(global_deadline)
  n_devices_whomadeit.append(devices_ok.count(1))
# print("Final deadline to obtain "+str(required_devices*100)+"% of devices into the model computation:"+str(global_deadline))
print("Percentage is: "+str(round(devices_ok.count(1)*100/len(devices_ok),2)) + "% " )
deadline_SP = global_deadline

Global deadline based on all the devices is: 1919.0
For the virtual deadline Devices who can make it: 42 VS who can't: 158
Percentage is: 21.0% 
Devices who can make it after 1 iteration : 60 VS who still can't: 140
Devices who can make it after 2 iteration : 76 VS who still can't: 124
Devices who can make it after 3 iteration : 85 VS who still can't: 115
Devices who can make it after 4 iteration : 98 VS who still can't: 102
Devices who can make it after 5 iteration : 108 VS who still can't: 92
Devices who can make it after 6 iteration : 122 VS who still can't: 78
Devices who can make it after 7 iteration : 123 VS who still can't: 77
Devices who can make it after 8 iteration : 130 VS who still can't: 70
Devices who can make it after 9 iteration : 140 VS who still can't: 60
Devices who can make it after 10 iteration : 143 VS who still can't: 57
Devices who can make it after 11 iteration : 147 VS who still can't: 53
Devices who can make it after 12 iteration : 150 VS who still can't: 50


In [39]:
optimalChoisesDelta = []
optimalCpu = []
batteryUsage = []
avgEnergy = []
for i in devices:
  optimalChoisesDelta.append(i.cpuFrequencyMax-i.optimalFrequency)
  optimalCpu.append(i.optimalFrequency)
  batteryUsage.append(i.EnergyConsumption)
print("avg Delta is: " +  str( round(sum(optimalChoisesDelta)/len(optimalChoisesDelta),2) ))
print("avg CPU is: " +  str( round(sum(optimalCpu)/len(optimalCpu),2))) 
avgEnergy.append(round(sum(batteryUsage)/len(batteryUsage),2))
print("avg Energy Consumption is: " +  str( avgEnergy[0]) )

avg Delta is: 0.34
avg CPU is: 0.57
avg Energy Consumption is: 3533.79


In [40]:
n_devices_whomadeit = []
devices_ok = []
iteration = 0
required_devices = 1.0
for i in devices:
  # print(i)
  i.set_deadline(global_deadline)
  devices_ok.append(i.setCpu(nData))

n_devices_whomadeit.append(devices_ok.count(1))
while(devices_ok.count(1) < required_devices * n_devices):
  iteration +=1
  devices_ok = []
  # Recalculate new deadline for next round a bit larger based on old deadline so to avoid useless computation
  global_deadline = deadline_determination(devices,nData,required_devices,global_deadline)
  for i in devices:
    # print(i)
    i.set_deadline(global_deadline)
    devices_ok.append(i.setCpu(nData))
  # print("Devices who can make it after "+str(iteration)+" iteration : "+str(devices_ok.count(1)) + " VS who still can't: " +str(devices_ok.count(0)) + " DEADLINE: "+ str(round(global_deadline,2))  )
  deadlines.append(global_deadline)
  n_devices_whomadeit.append(devices_ok.count(1))
print("Final deadline to obtain "+str(required_devices*100)+"% of devices into the model computation:"+str(global_deadline))
deadline_classic = global_deadline

Final deadline to obtain 100.0% of devices into the model computation:20196.0


In [41]:
optimalChoisesDelta = []
optimalCpu = []
batteryUsage = []
for i in devices:
  optimalChoisesDelta.append(i.cpuFrequencyMax-i.optimalFrequency)
  optimalCpu.append(i.optimalFrequency)
  batteryUsage.append(i.EnergyConsumption)
print("avg CPU is: " +  str( round(sum(optimalCpu)/len(optimalCpu),2))) 
avgEnergy.append(round(sum(batteryUsage)/len(batteryUsage),2))
print("avg Energy Consumption is: " +  str( avgEnergy[1]) )

avg CPU is: 0.92
avg Energy Consumption is: 16722.98


In [42]:
print("SMART PC AVG ENERGY CONSUMPTION:" + str(round(avgEnergy[0],2)) )
print("CLASSIC APPROACH AVG ENERGY CONSUMPTION:" + str(round(avgEnergy[1],2)))
print("SMARTPC performs "+str(round(100*avgEnergy[0]/(avgEnergy[1]),2)) +"% better than classical approach")
print("#############################################################################################################")
print("SMART PC DEADLINE:" + str(round(deadline_SP,2)) )
print("CLASSIC APPROACH DEADLINE:" + str(round(deadline_classic,2)))
print("SMARTPC performs "+str(round(100*deadline_SP/deadline_classic,2))  +"% better than classical approach")


SMART PC AVG ENERGY CONSUMPTION:3533.79
CLASSIC APPROACH AVG ENERGY CONSUMPTION:16722.98
SMARTPC performs 21.13% better than classical approach
#############################################################################################################
SMART PC DEADLINE:6768.0
CLASSIC APPROACH DEADLINE:20196.0
SMARTPC performs 33.51% better than classical approach
